## 00_read.ipynb

- Reading and executing notebooks. 
  - A notebook is a json.Eg: `Path(path).read_json()`
  - Convert the json into format suitable for nbclient (instead of nbformat), we are using fastcore `dict2obj`.
  - read_nb 
- Config
  - nbprocess_create_config # Creates settings.ini using `locals()` passed to the func (wrapper for fastcore `save_config_file`).
- Exporting a basic module
  - Python module requires a `__init__.py` file in all directories considered as modules.
- Export
  - write cells to file along with hdr starting at index offset
  - basic exporter to bootstrap nbprocess
    * grab source from all cells in nb containing `export` comment
    * find all the exported funcs to create `__all__` 
    * exec_new -> Execute `code` in a new environment and return it
- tip: use `from pdb import set_trace()`

## 01_maker.ipynb

- str.splitlines -> Split str into a list of lines breaking at line boundaries.
- Variable helpers -> find_var read_var update_var
- ModuleMaker - Helper class to create exported library from notebook source cells
  - make_all -> create `__all__` with all exports in cells.
  - make -> write module containing cells with `__all__` generated from all_cells. Pass `all_cells=[]` if no `__all__`.
  - _update_all
  - _make_exists -> make for is_new=False, additional defn are added to the bottom.
- make_code_cell make_code_cells
- basic_export_nb2 -> `ModuleMaker(dest/name).make(read_nb(fname).cells)`

## 02_export.ipynb

- NBProcessor -> Special comments at the start of a cell can be used to provide information to nbprocess about how to process a cell, so we need to be able to find the location of these comments.
  - `extract_comments` -> nbprocess comments start with #, followed by whitespace delimited tokens, which extract_comments extracts from the start of a cell, up until a blank line or a line containing something other than comments:
  - _param_count -> Get no. of params accepted by func 'f' using `from inspect import signature,Parameter`
  -  `NBProcessor(nb_fname, procs=Processor, debug=True)` -> Process all cells with `process_cell`
- From `01_everything.ipynb`
```python
#printme testing
_tmp = "Cell for testing processor subclass"
```

```python -> - Return testing
class _PrintExample:
    def _printme_(self, nbp, to_print): print(to_print)

everything_fn = '../tests/01_everything.ipynb'
proc = NBProcessor(everything_fn, _PrintExample())
proc.process()
```

- ExportModuleProc -> Processor exporting code to a module.
  - Exported cells are stored in a dict called modules, where the keys are the modules exported to. Those without an explicit module are stored in the '#' key, which will be exported to default_exp.
```python
class ExportModuleProc:
    def __init__(self): ...
    def _default_exp_(self, nbp, exp_to): ...
    def _exporti_(self, nbp, exp_to=None): ...
    def _export_(self, nbp, exp_to=None): ...
```
- create_modules(path, dest, procs=[ExportModuleProc(), rm_comments_proc], debug=True) -> create modules from nb.
- nb_export